# Comments to the following code
> - #### Parts 1 and 2 can be run separately
> - #### Raw data: Excel sheet can be obtained from https://doi.org/10.1021/acs.est.8b01452
> - #### Terms `exchange` and `input activity` are used interchangeably
> - #### At the moment, only works if ecoinvent 3.3 was also imported as reference product is taken from there

# ------------------------------------------ Part 1 ------------------------------------------

In [1]:
import pandas as pd
import numpy as np
from copy import copy
import os, json
import re
import brightway2 as bw

# Local files
from utils_consumption_db import *

# Convert data to brightway database format -> main code
calls all the functions used above

In [2]:
with open('global_settings.json', 'rb') as f:
    settings = json.load(f)
which_pc = settings['which_pc']

In [3]:
%%time
# 0. Set relevant paths
if which_pc == 'local':
    path_habe = '/Users/akim/Documents/LCA_files/HABE_2017/'
    ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
elif which_pc == 'merlin':
    path_habe = '/data/user/kim_a/LCA_files/HABE_2017'
    ei33_path = '/data/user/kim_a/LCA_files/ecoinvent_33_cutoff/datasets'  
    
# 1. Create project with ecoinvent 3.3 cutoff only. 
# This is needed because Andi's activities were given as ecoinvent 3.3 cutoff codes
create_ecoinvent_33_project(ei33_path)

# Start brightway project that already contains databases
project = 'GSA for ecoinvent'
bw.projects.set_current(project)

# Create dataframe that will be our consumption database after we add activities and exchanges from the raw file
df_bw = create_df_bw(CONSUMPTION_DB_NAME)

# Read data from Andi's consumption database
path = 'data/es8b01452_si_002.xlsx'
sheet_name = 'Overview & LCA-Modeling'
df_raw = pd.read_excel(path, sheet_name = sheet_name, header=2)

# Read data from HABE
year = '091011' #121314 or 151617
code_unit = get_units_habe(path_habe, year)

# Add ON columns
df = complete_columns(df_raw)

act_indices = df_raw.index[df_raw['ConversionDem2FU'].notna()].tolist() # indices of all activities

for ind in act_indices:
    # For each row
    df_ind = df_raw.iloc[ind]
    df_ind = df_ind[df_ind.notna()]
    # Add activity
    df_bw, df_act = append_activity(df_bw, df_ind[:N_ACT_RELEVANT], code_unit) # only pass columns relevant to this function 
    # Add exchanges
    df_bw = append_exchanges(df_bw, df_ind, df_act, append_one_exchange=append_ecoinvent_exchange)

Biosphere database already present!!! No setup is needed
ecoinvent 3.3 cutoff database already present!!! No import is needed
CPU times: user 57.8 s, sys: 8.23 s, total: 1min 6s
Wall time: 3min 1s


In [4]:
# Write the dataframe to excel file
write_dir_name = 'write_files'
if not os.path.exists(write_dir_name):
    os.mkdir(write_dir_name)
db_bw_path = write_dir_name + '/' + 'consumption_db.xlsx'
df_bw.to_excel(db_bw_path, index=False, header=False)

# ------------------------------------------ Part 2 ------------------------------------------

In [5]:
import pandas as pd
import numpy as np
import brightway2 as bw
import string
from copy import copy, deepcopy

# Local files
from utils import *
from utils_consumption_db import *

In [6]:
# Main code
project = 'GSA for ecoinvent'
bw.projects.set_current(project)

# Read consumption database
path = 'write_files/consumption_db.xlsx'
df = pd.read_excel(path, header = None)
df.columns = list(string.ascii_uppercase[:len(df.columns)])
# 
# Replace
df = update_all_db(df)
path_new_db = 'write_files/consumption_db_updated.xlsx'
df.to_excel(path_new_db, index=False, header=False)

# Import consumption database linked to older versions of other databases

# 1. Ecoinvent 3.6

### TODO Chris -> please check migrations

In [7]:
if CONSUMPTION_DB_NAME in bw.databases:
    del bw.databases[CONSUMPTION_DB_NAME]

In [8]:
co = bw.ExcelImporter(path_new_db)
co.apply_strategies()
co.match_database('EXIOBASE 2.2', fields=('name','reference product', 'unit','location','categories'))
co.match_database('ecoinvent 3.6 cutoff', fields=('name', 'reference product', 'unit','location','categories'))
co.match_database('Agribalyse 1.3 - ecoinvent 3.6 cutoff', fields=('name','unit','location'))
co.statistics()

Extracted 1 worksheets in 0.12 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.42 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
203 datasets
834 exchanges
95 unlinked excha

(203, 834, 95)

In [9]:
# Define a migration for two particular activities that can only be hardcoded
ecoinvent36_change_names_data = {
    'fields': ['name', ],
    'data': [
        (
            ['steam production in chemical industry'], 
            {
                'name': 'steam production, in chemical industry',
                'reference product': 'steam, in chemical industry',
                'unit': 'kilogram',
                'multiplier': 1/2.75, # see comment on this activity in ecoinvent
            }
        ),
        (
            ['market for green bell pepper'],
            {
                'name': 'market for bell pepper',
                'reference product': 'bell pepper',
            }
        ),
    ]
}

bw.Migration("ecoinvent36-change-names").write(
    ecoinvent36_change_names_data,
    description="Change names of some activities"
)

In [10]:
# Define a migration for rice production and specific locations
# These locations have only non-basmati rice production
ecoinvent36_rice_production_data = {
    'fields': ['name', 'location'],
    'data': [
        (
            ['rice production', 'US'],
            {
                'name': 'rice production, non-basmati',
                'reference product': 'rice, non-basmati'
            }
        ),
        (
            ['rice production', 'CN'],
            {
                'name': 'rice production, non-basmati',
                'reference product': 'rice, non-basmati'
            }
        ),
    ]
}

bw.Migration("ecoinvent36-rice-production").write(
    ecoinvent36_rice_production_data,
    description="Change names of some activities"
)

In [11]:
co.migrate('ecoinvent36-change-names')
co.migrate("ecoinvent36-rice-production")
co.match_database('ecoinvent 3.6 cutoff', fields=('name','reference product', 'unit','location','categories'))
co.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
203 datasets
834 exchanges
26 unlinked exchanges
  Type technosphere: 7 unique unlinked exchanges


(203, 834, 26)

The rest of the unlinked exchanges are not uniquely defined in ecoinvent 3.6 -> 1-to-multiple mapping. <br>
For example 'rice production' is now divided into basmati and non-basmati rice. <br>
Hence, we split them based on their shares in the production volumes.

In [12]:
# Manually choose which ecoinvent 3.6 exchanges should be taken for each unlinked exchange
ei36 = bw.Database('ecoinvent 3.6 cutoff')
mapping = [
    {('market for rice', 'GLO'): 
        [act['code'] for act in ei36 if  'market for rice' in act['name'] 
                                     and act['location']=='GLO'
                                     and 'seed' not in act['name']]},
    
    {('rice production', 'RoW'): 
        [act['code'] for act in ei36 if  'rice production' in act['name'] 
                                     and act['location']=='RoW'
                                     and 'straw' not in act['reference product']]},
    
    {('rice production', 'IN'): 
        [act['code'] for act in ei36 if  'rice production' in act['name'] 
                                     and act['location']=='IN'
                                     and 'straw' not in act['reference product']]},
    
    {('market for wheat grain', 'GLO'): 
        [act['code'] for act in ei36 if  'market for wheat grain' in act['name'] 
                                     and 'feed' not in act['name']]},
    
    {('market for maize grain', 'GLO'): 
        [act['code'] for act in ei36 if  'market for maize grain' in act['name'] 
                                     and 'feed' not in act['name']]},
    
    {('market for mandarin', 'GLO'): 
        [act['code'] for act in ei36 if 'market for mandarin' in act['name']]},
    
    {('market for soybean', 'GLO'): 
        [act['code'] for act in ei36 if 'market for soybean' in act['name'] 
                             and all([_ not in act['name'] for _ in ['meal','beverage','seed','feed','oil']] )]},
]

In [13]:
ei_name = 'ecoinvent 3.6 cutoff'

In [14]:
co = modify_exchanges(co, mapping, 'ecoinvent 3.6 cutoff')
co.statistics()

Applying strategy: link_iterable_by_fields
203 datasets
895 exchanges
0 unlinked exchanges
  


(203, 895, 0)

In [15]:
if CONSUMPTION_DB_NAME in bw.databases:
    print(CONSUMPTION_DB_NAME + " database already present!!! No import is needed")
else:
    co.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/07/2020 12:27:04
  Finished: 07/07/2020 12:27:04
  Total time elapsed: 00:00:00
  CPU %: 60.90
  Memory %: 0.06
Created database: CH consumption 1.0


In [16]:
bw.databases

Databases dictionary with 6 object(s):
	Agribalyse 1.3 - ecoinvent 3.6 cutoff
	Agribalyse 1.3 - ecoinvent 3.6 cutoff - new biosphere
	CH consumption 1.0
	EXIOBASE 2.2
	biosphere3
	ecoinvent 3.6 cutoff

# ------------------------------------------ Part 3 ------------------------------------------

In [17]:
import brightway2 as bw
import numpy as np
import pandas as pd
import os

In [18]:
bw.projects.set_current('GSA for ecoinvent')

In [19]:
bw.databases

Databases dictionary with 6 object(s):
	Agribalyse 1.3 - ecoinvent 3.6 cutoff
	Agribalyse 1.3 - ecoinvent 3.6 cutoff - new biosphere
	CH consumption 1.0
	EXIOBASE 2.2
	biosphere3
	ecoinvent 3.6 cutoff

In [20]:
co = bw.Database('CH consumption 1.0')

# Add consumption activity for all households

## 1. Extract total demand from HABE

In [21]:
if which_pc == 'local':
    path = '/Users/akim/Documents/LCA_files/HABE_2017/'
elif which_pc == 'merlin':
    path = '/data/user/kim_a/LCA_files/HABE_2017'
    
year = '091011' #11, 14 or 17
get_path = lambda which_file: os.path.join( path, [f for f in os.listdir(path) if year in f and which_file in f][0] )
path_beschrei = get_path('Datenbeschreibung')
path_ausgaben = get_path('Ausgaben')

In [22]:
# change codes to be consistent with consumption database and Andi's codes
bw.projects.set_current('GSA for ecoinvent')
co = bw.Database('CH consumption 1.0')

ausgaben = pd.read_csv(path_ausgaben, sep='\t')

codes_co_db = sorted([act['code'] for act in co])
columns_a = ausgaben.columns.values
columns_m = [columns_a[0]]
for code_a in columns_a[1:]:
    code_m = code_a.replace('A', 'm') 
    if code_m in codes_co_db:
        columns_m.append(code_m)
    else:
        columns_m.append(code_a.lower())
        
ausgaben.columns = columns_m

In [23]:
# Compute total consumption
total_consumption = ausgaben.sum()
total_consumption = total_consumption.drop('HaushaltID')

In [24]:
# Add other useful info, eg number of households and number of people
meta = pd.read_excel(path_beschrei, sheet_name='Tabellen', skiprows=8, usecols=[0,1,3,4])
meta.columns = ['category1', 'category2', 'n_rows', 'n_cols']
meta.dropna(subset=['n_rows'], inplace=True)

# Combine some columns together
temp1 = meta[meta['category1'].notnull()][['category1', 'n_rows', 'n_cols']]
temp1.columns = ['category2', 'n_rows', 'n_cols']
temp2 = meta[meta['category2'].notnull()][['category2', 'n_rows', 'n_cols']]
meta = pd.concat([temp1, temp2])
meta.set_index('category2', inplace=True)

# Add info
total_consumption['n_households'] = meta.loc['HABE091011_Ausgaben']['n_rows']
total_consumption['n_people']     = meta.loc['HABE091011_Personen']['n_rows']

In [25]:
# Save total demand
path_demand = 'write_files/habe_totaldemands.xlsx'
total_consumption.to_excel(path_demand)

In [26]:
total_consumption['a5220']

354306.4719774653

#### OPTION 1. Total demands extract directly from HABE raw files
Excel file `habe_totaldemands.xlsx` contains sums of all private households in Switzerland for all categories of the HBS. Units are the same as in the HBS (please refer to the SI-excel of Andi's ES&T-paper in order to translate the codenames). The attached vector is in "per month" quantities.

#### OPTION 2. Andi's total demands from his Swiss consumption model
Excel file `heia2_totaldemands.xlsx` contains sums of all private households in Switzerland for all categories of the HBS. Please note that the units are basically the same as in the HBS (please refer to the SI-excel of Andi's ES&T-paper in order to translate the codenames). However, the attached vector is in "per year" instead of in "per month". Furthermore, there are a couple of demands that were computed by the model itself. The codenames for these computed/imputed categories start with "mx" and the units are as follows:
* kWh per year for electricity
* MJ per year for heating
* cubic meters per year for water supply and wastewater collection
* number of waste bags per year for refuse collection

In [27]:
option = 'aggregated'
if option == 'aggregated':
#     path = 'data/heia2_totaldemands.xlsx'm # from Andi's model
#     number_households = 3518878
    path = 'write_files/habe_totaldemands.xlsx'
    df   = pd.read_excel(path)
    df.columns = ['code', 'amount']
    df.set_index('code', inplace=True)

    n_households = int(df.loc['n_households', 'amount'])
    n_people     = int(df.loc['n_people', 'amount'])
    df = df.drop(['n_households', 'n_people'])
    df = df.reset_index()
    
elif option == 'disaggregated':
    path = 'data/habe20092011_hh_prepared_imputed.csv'
    df = pd.read_csv(path, low_memory=False)
    number_households = df.shape[0]
    df = df.drop('haushaltid', axis=1).sum()
    df = df.reset_index()
    df.columns = ['code', 'amount']

# for i in range(len(df)):
#     code = df.iloc[i]['code']
#     new_code = code
#     df.at[i,'code'] = new_code

In [28]:
# Add total inputs from Andi's model as swiss consumption activity
try: co.get('ch hh all consumption').delete()
except: pass
consumption_all = co.new_activity('ch hh all consumption', name='ch hh all consumption', location='CH', unit='1 month of consumption')
consumption_all.save()

In [29]:
# Add production exchange for the activity `consumption`
consumption_all.new_exchange(input = (consumption_all['database'], consumption_all['code']),
                         amount = 1,
                         type = 'production').save()

In [30]:
codes = [act['code'] for act in co]

unlinked_codes = []
for i in range(len(df)):
    code = df.loc[i]['code']
    if code in codes:
        consumption_all.new_exchange(input  = (co.name, code), 
                                     amount = df.loc[i]['amount'], # TODO?? divide by number of months
                                     type   = 'technosphere').save()
    else:
        unlinked_codes.append(code)

In [31]:
list(consumption_all.exchanges()) # per month for all households

[Exchange: 1 1 month of consumption 'ch hh all consumption' (1 month of consumption, CH, None) to 'ch hh all consumption' (1 month of consumption, CH, None)>,
 Exchange: 1961093.913276128 CHF 'Other insurance premiums' (CHF, CH, None) to 'ch hh all consumption' (1 month of consumption, CH, None)>,
 Exchange: 745048.1685118991 CHF 'Fees' (CHF, CH, None) to 'ch hh all consumption' (1 month of consumption, CH, None)>,
 Exchange: 25817.66898981574 kilogram 'Rice' (kilogram, CH, None) to 'ch hh all consumption' (1 month of consumption, CH, None)>,
 Exchange: 99858.28575933525 kilogram 'Pasta products' (kilogram, CH, None) to 'ch hh all consumption' (1 month of consumption, CH, None)>,
 Exchange: 303400.7717455065 kilogram 'Bread' (kilogram, CH, None) to 'ch hh all consumption' (1 month of consumption, CH, None)>,
 Exchange: 448103.1234104095 kilogram 'Bakery products (sweet and salty)' (kilogram, CH, None) to 'ch hh all consumption' (1 month of consumption, CH, None)>,
 Exchange: 28832.4549

In [32]:
len(list(consumption_all.exchanges()))

195

### Note that the number of consumption exchanges is the same as the number of activities in the database, but is a lot less than what Andi provided in his total demands.

# Add consumption activity for an average household

In [33]:
try: co.get('ch hh average consumption').delete()
except: pass
consumption_average = consumption_all.copy('ch hh average consumption', name='ch hh average consumption')

In [34]:
for exc in consumption_average.exchanges():
    if exc['type'] != 'production':
        exc['amount'] /= n_households
        exc.save()

In [35]:
list(consumption_average.exchanges())

[Exchange: 1 1 month of consumption 'ch hh average consumption' (1 month of consumption, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 201.46845215493406 CHF 'Other insurance premiums' (CHF, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 76.54080218942872 CHF 'Fees' (CHF, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 2.6523185730240124 kilogram 'Rice' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 10.258710269091356 kilogram 'Pasta products' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 31.169177290477347 kilogram 'Bread' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 46.03483906003796 kilogram 'Bakery products (sweet and salty)' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumptio

# Simple LCA

In [36]:
import brightway2 as bw

In [37]:
bw.projects.set_current('GSA for ecoinvent')

In [38]:
co = bw.Database('CH consumption 1.0')

In [39]:
consumption_average = co.search('average consumption')[0]

In [40]:
# demand = {consumption_average: 1}
demand = {consumption_average: 1}
method = ('IPCC 2013', 'climate change', 'GTP 100a')

In [41]:
%%time
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
print(str(lca.score) + ' ' + bw.Method(method).metadata['unit'])

3112.3489740053456 kg CO2-Eq
CPU times: user 2.12 s, sys: 80.4 ms, total: 2.2 s
Wall time: 2.21 s


### Another exchange

In [42]:
food = co.search('Ready-made foods')[0]

In [43]:
%%time
lca = bw.LCA({food: 1}, method)
lca.lci()
lca.lcia()
print(str(lca.score) + ' ' + bw.Method(method).metadata['unit'])

0.29370403056054556 kg CO2-Eq
CPU times: user 384 ms, sys: 38 ms, total: 422 ms
Wall time: 268 ms


# ------------------------------------------ Part 4 ------------------------------------------

# Add categories for all activities (in English)

In [ ]:
import brightway2 as bw
import numpy as np
import pandas as pd
import re

In [ ]:
bw.projects.set_current('GSA for ecoinvent')

In [ ]:
path = 'data/es8b01452_si_002.xlsx'
sheet_name = 'Overview & LCA-Modeling'
df_raw = pd.read_excel(path, sheet_name = sheet_name, header=2)

In [ ]:
categories_col_de = 'Original name in Swiss household budget survey'
categories_col_en = 'Translated name'
categories_raw = df_raw[[categories_col_de, categories_col_en]]

In [ ]:
categories_raw.values

In [ ]:
categories = {}
for v in categories_raw.values:
    v_list_de = v[0].split(':')
    v_list_en = v[1].split(':')
    if len(v_list_de)>1 and len(v_list_de[0].split('.')) == 1:
        categories[v_list_de[0]] = v_list_en[0]
max_code_len = max({len(k) for k in categories.keys()})

In [ ]:
co = bw.Database('CH consumption 1.0')

In [ ]:
%%time
category_names_dict = {
    2: 'coarse',
    3: 'middle',
    4: 'fine',
}
for act in co:    
    code = re.sub(r'[a-z]+', '', act['code'], re.I)[:max_code_len]
    for i in range(2,max_code_len+1):
        try:
            category_name = 'category_' + category_names_dict[i]
            act[category_name] = categories[code[:i]]
            act.save()
        except:
            pass

In [ ]:
co.random().as_dict()

# ------------------------------------------ Part 5 ------------------------------------------

# Add sectors as separate activities
## 1. Choose sectors

In [ ]:
import brightway2 as bw
import pandas as pd

In [ ]:
bw.projects.set_current('GSA for ecoinvent')

In [ ]:
co = bw.Database('CH consumption 1.0')

In [ ]:
cat_option = 'category_coarse'

cat_unique = []
for act in co:
    cat_unique.append(act.get(cat_option) or 0)
cat_unique = list(set(cat_unique))

category_activities = {}
category_activities_len = {}
for cat_of_interest in cat_unique: 
    list_ = []
    for act in co:
        if act.get(cat_option) == cat_of_interest:
            list_.append(act)
    if len(list_) > 0:
        category_activities[cat_of_interest] = list_
        category_activities_len[cat_of_interest] = len(list_)

In [ ]:
dict_ = {}
for cat_of_interest, activities in category_activities.items():
        
    excs_input_ag  = []
    excs_input_ex  = []
    excs_input_ec  = []

    for act in activities:
        for exc in act.exchanges():
            if 'Agribalyse' in exc.input['database']:
                excs_input_ag.append(exc.input)
            elif 'EXIOBASE' in exc.input['database']:
                excs_input_ex.append(exc.input)
            elif 'ecoinvent 3.6 cutoff' == exc.input['database']:
                excs_input_ec.append(exc.input)
                
    dict_[cat_of_interest] = dict(
        n_activities = len(activities),
        n_agribalyse_exchanges = len(excs_input_ag), 
        n_exiobase_exchanges   = len(excs_input_ex), 
        n_ecoinvent_exchanges  = len(excs_input_ec), 
    )
        

In [ ]:
df = pd.DataFrame.from_dict(dict_).T

In [ ]:
category_activities_len

## 2. Add sectors

In [ ]:
demand_act = [act for act in co if 'average' in act['name']][0]
demand_act

In [ ]:
for cat_of_interest in category_activities:
    # Create new bw activity with a specific name
    try: co.get(cat_of_interest).delete()
    except: pass
    new_act = co.new_activity(cat_of_interest, 
                              name=cat_of_interest, 
                              location='CH', 
                              unit='1 month of consumption',
                              comment='Average consumption of one household',
                             )
    new_act.save()
    
    # Add production exchange
    new_act.new_exchange(
        input = (new_act['database'], new_act['code']),
        amount = 1,
        type = 'production'
    ).save()
    
    for exc in demand_act.exchanges():
        if exc.input.get('category_coarse')==cat_of_interest:
            new_act.new_exchange(
                input  = (exc.input['database'], exc.input['code']),
                amount = exc.amount,
                type   = 'technosphere'
            ).save()


## 3. Contribution analysis for sectors

In [ ]:
list_methods = [('IPCC 2013', 'climate change', 'GTP 100a')]
list_fus = []
for cat_of_interest in category_activities:
    list_fus.append({co.search(cat_of_interest)[0]: 1})

In [ ]:
bw.calculation_setups['sector_contribution_analysis'] = {'inv':list_fus, 'ia':list_methods}

In [ ]:
%%time
myMultiLCA = bw.MultiLCA('sector_contribution_analysis')

In [ ]:
lcia_unit = bw.Method(list_methods[0]).metadata['unit']

fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_fus]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_fus]

In [ ]:
df.sort_values(lcia_unit, ascending=False, inplace=True)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)